<a href="https://colab.research.google.com/github/RansomwareDetection/RamsomwareTrafficDetection/blob/colab/Updated_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load the dataset
data = pd.read_csv("/content/Android_Ransomeware.csv", on_bad_lines='skip')

# Step 1: Filter dataset to include only "Benign" and "RansomBO" samples
data = data[data['Label'].isin(['Benign', 'RansomBO'])]

# Step 2: Convert 'Label' to 0 for 'Benign' and 1 for 'RansomBO'
data['Label'] = data['Label'].apply(lambda x: 0 if x == 'Benign' else 1)

# Step 3: Select the relevant network traffic columns
selected_columns = [
    ' ACK Flag Count', ' Fwd Header Length.1', ' Avg Bwd Segment Size', ' Avg Fwd Segment Size',
    ' Average Packet Size', ' Down/Up Ratio', ' ECE Flag Count', ' CWE Flag Count', ' URG Flag Count',
    ' PSH Flag Count', 'Fwd Packets/s', ' RST Flag Count', ' SYN Flag Count', 'FIN Flag Count',
    ' Packet Length Variance', ' Packet Length Std', ' Packet Length Mean', ' Max Packet Length',
    ' Min Packet Length', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate',
    ' Bwd Avg Bytes/Bulk', ' Idle Max', ' Idle Std', 'Idle Mean', ' Active Min', ' Active Max'
]

# Step 4: Separate features and target
X = data[selected_columns]
y = data['Label']

# Step 5: Convert all columns to numeric and handle missing values
X = X.apply(pd.to_numeric, errors='coerce').fillna(X.mean())

# Step 6: Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# Step 7: Reshape input data to fit CNN (batch_size, channels, height, width)
X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1], 1)

# Step 9: Define the CNN + GRU Model
class MalwareDetectionCNNGRU(nn.Module):
    def __init__(self):
        super(MalwareDetectionCNNGRU, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 1))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 1))
        self.relu = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(p=0.4)

        # Calculate GRU input size
        self.gru_input_size = self._get_flatten_size(torch.zeros(1, 1, X_train_tensor.shape[2], 1))
        self.gru = nn.GRU(input_size=self.gru_input_size, hidden_size=128, num_layers=2, batch_first=True, dropout=0.4)

        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 2)

    def _get_flatten_size(self, sample_input):
        with torch.no_grad():
            x = self.pool(self.batchnorm1(self.relu(self.conv1(sample_input))))
            x = self.pool(self.batchnorm2(self.relu(self.conv2(x))))
            x = x.view(x.size(0), -1)
        return x.size(1)

    def forward(self, x):
        x = self.pool(self.batchnorm1(self.relu(self.conv1(x))))
        x = self.pool(self.batchnorm2(self.relu(self.conv2(x))))
        x = x.view(x.size(0), 1, -1)  # Reshape for GRU (batch, seq_len=1, features)
        x, _ = self.gru(x)  # Pass through GRU
        x = x[:, -1, :]  # Get the last hidden state
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


# Step 10: Set up model, optimizer, and loss function
model = MalwareDetectionCNNGRU()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Move data and model to device (if using GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X_train_tensor, X_test_tensor = X_train_tensor.to(device), X_test_tensor.to(device)
y_train_tensor, y_test_tensor = y_train_tensor.to(device), y_test_tensor.to(device)

# Step 11: Train the model with early stopping
num_epochs = 100
best_accuracy = 0
early_stop_patience = 5
patience_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        predicted = torch.argmax(test_outputs, dim=1)
        accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)

        # Early stopping
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter > early_stop_patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Test Accuracy: {accuracy * 100:.2f}%')

# Step 12: Evaluate performance with a confusion matrix
cm = confusion_matrix(y_test_tensor.cpu(), predicted.cpu())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Benign', 'Ransomware'])
disp.plot()
plt.show()

print(f"Best Test Accuracy: {best_accuracy * 100:.2f}%")


ParserError: Error tokenizing data. C error: Expected 86 fields in line 6019, saw 135
